In [36]:
import requests
import pandas as pd
from io import StringIO
import geopandas as gpd
import datetime

baseurl = "https://beachwatch.waterboards.ca.gov/public/"
reports_page=f"{baseurl}advisory.php"
exports_page=f"{baseurl}export.php"

formdata={
    "type":"3", # closure
   "County":10,
"stationID":"",
"cause":"",
"source":"",
"substance":"",
"created":"",
"year":"2025",
"sort":"`Start Date`",
"sortOrder":"DESC",
"submit":"Search ",
}

To Get Closure information over time:

https://www.waterboards.ca.gov/water_issues/programs/beaches/search_beach_advisory.html

first page after search: with export to excel

https://beachwatch.waterboards.ca.gov/public/advisory.php

In [37]:
response = requests.post(reports_page, data=formdata)
cookies = response.cookies

response = requests.get(exports_page, cookies=cookies)   
data = response.text

beach_df = pd.read_csv(StringIO(data), sep="\t")
beach_df.head()

,id,Description,StationDescription,Beach Name,Latitude,Longitude,Extent Start,Extent Start Type,Extent End,Extent End Type,...,Start Date,Start Time,End Date,End Time,County,Comments,Enterococcus,Fecal Coliforms,Total Coliforms,Area Description
0,65599,NaN,Navy Fence (A),Coronado north beach,32.685586,-117.194319,150.0,Feet Downcoast,150.0,Feet Upcoast,...,2025-03-22,16:35,2025-03-23,15:56,San Diego,NaN,0,0,0,NaN
1,65567,NaN,Vista De La Playa,Marine Street Beach,32.834100,-117.281500,150.0,Feet Downcoast,150.0,Feet Upcoast,...,2025-03-15,18:49,2025-03-17,14:00,San Diego,NaN,0,0,0,NaN
2,65565,NaN,Avenida Lunar,Coronado City beaches,32.674066,-117.172300,150.0,Feet Downcoast,150.0,Feet Upcoast,...,2025-03-14,11:00,2025-03-23,15:56,San Diego,NaN,0,0,0,NaN
3,65564,NaN,Navy Fence (A),Coronado north beach,32.685586,-117.194319,150.0,Feet Downcoast,150.0,Feet Upcoast,...,2025-03-14,11:00,2025-03-21,17:04,San Diego,NaN,0,0,0,NaN
4,65563,NaN,Loma Ave (frmrly Isabella,Coronado City beaches,32.681840,-117.185260,150.0,Feet Downcoast,150.0,Feet Upcoast,...,2025-03-14,11:00,2025-03-23,15:56,San Diego,NaN,0,0,0,NaN


In [38]:
#beach_df['date']=  pd.to_datetime(f'{beach_df["SampleDate"]}T{beach_df["SampleTime"]}')#, format='%Y-%m-%d %H:%M:%S')
beach_df['start']=  pd.to_datetime(beach_df["Start Date"], format='%Y-%m-%d', errors='coerce')
beach_df['end']=pd.to_datetime(beach_df["End Date"], format='%Y-%m-%d', errors='coerce')

beach_df['end']= beach_df['end'].fillna(value=datetime.datetime.now())
beach_df.dropna(subset=['start'], inplace=True)
gs=gpd.GeoSeries.from_xy(beach_df['Longitude'],beach_df['Latitude'])
beach_gdf= gpd.GeoDataFrame(beach_df, 
    geometry=gs ,
    crs = 'EPSG:4326')


beach_gdf['Icon']= 'place'
beach_gdf

,id,Description,StationDescription,Beach Name,Latitude,Longitude,Extent Start,Extent Start Type,Extent End,Extent End Type,...,County,Comments,Enterococcus,Fecal Coliforms,Total Coliforms,Area Description,start,end,geometry,Icon
0,65599,NaN,Navy Fence (A),Coronado north beach,32.685586,-117.194319,150.00,Feet Downcoast,150.00,Feet Upcoast,...,San Diego,NaN,0,0,0,NaN,2025-03-22,2025-03-23 00:00:00.000000,POINT (-117.19432 32.68559),place
1,65567,NaN,Vista De La Playa,Marine Street Beach,32.834100,-117.281500,150.00,Feet Downcoast,150.00,Feet Upcoast,...,San Diego,NaN,0,0,0,NaN,2025-03-15,2025-03-17 00:00:00.000000,POINT (-117.2815 32.8341),place
2,65565,NaN,Avenida Lunar,Coronado City beaches,32.674066,-117.172300,150.00,Feet Downcoast,150.00,Feet Upcoast,...,San Diego,NaN,0,0,0,NaN,2025-03-14,2025-03-23 00:00:00.000000,POINT (-117.1723 32.67407),place
3,65564,NaN,Navy Fence (A),Coronado north beach,32.685586,-117.194319,150.00,Feet Downcoast,150.00,Feet Upcoast,...,San Diego,NaN,0,0,0,NaN,2025-03-14,2025-03-21 00:00:00.000000,POINT (-117.19432 32.68559),place
4,65563,NaN,Loma Ave (frmrly Isabella,Coronado City beaches,32.681840,-117.185260,150.00,Feet Downcoast,150.00,Feet Upcoast,...,San Diego,NaN,0,0,0,NaN,2025-03-14,2025-03-23 00:00:00.000000,POINT (-117.18526 32.68184),place
5,65363,NaN,Navy Fence (A),Coronado north beach,32.685586,-117.194319,150.00,Feet Downcoast,150.00,Feet Upcoast,...,San Diego,NaN,0,0,0,NaN,2025-02-13,2025-02-21 00:00:00.000000,POINT (-117.19432 32.68559),place
6,65362,NaN,Loma Ave (frmrly Isabella,Coronado City beaches,32.681840,-117.185260,150.00,Feet Downcoast,150.00,Feet Upcoast,...,San Diego,NaN,0,0,0,NaN,2025-02-13,2025-02-23 00:00:00.000000,POINT (-117.18526 32.68184),place
7,65361,NaN,Avenida Lunar,Coronado City beaches,32.674066,-117.172300,150.00,Feet Downcoast,150.00,Feet Upcoast,...,San Diego,NaN,0,0,0,NaN,2025-02-13,2025-02-23 00:00:00.000000,POINT (-117.1723 32.67407),place
8,65355,NaN,Lawrence St. outlet,San Diego Bay,32.709209,-117.236837,150.00,Feet Downcoast,150.00,Feet Upcoast,...,San Diego,NaN,0,0,0,Jet fuel in water due to aviation crash in San...,2025-02-12,2025-02-14 00:00:00.000000,POINT (-117.23684 32.70921),place
9,65321,NaN,Avenida Lunar,Coronado City beaches,32.674066,-117.172300,150.00,Feet Downcoast,150.00,Feet Upcoast,...,San Diego,NaN,1,0,0,NaN,2025-02-07,2025-02-08 00:00:00.000000,POINT (-117.1723 32.67407),place


In [39]:
def weeks_spanned(row):
    # Generate a date_range from start to end with a weekly frequency
    # Here, 'W-MON' gives the week starting on Monday.
    start_week = row['start'] - pd.to_timedelta(row['start'].weekday(), unit='D')
    end_week = row['end'] - pd.to_timedelta(row['end'].weekday(), unit='D')
    # Create a weekly date_range with a step of 7 days.
    return pd.date_range(start=start_week, end=end_week, freq='7D')

# Apply the function to create a new column with the weeks each event spans
beach_gdf['week'] = beach_gdf.apply(weeks_spanned, axis=1)

# Explode the list of weeks into individual rows
df_exploded = beach_gdf.explode('week')
df_exploded 

,id,Description,StationDescription,Beach Name,Latitude,Longitude,Extent Start,Extent Start Type,Extent End,Extent End Type,...,Comments,Enterococcus,Fecal Coliforms,Total Coliforms,Area Description,start,end,geometry,Icon,week
0,65599,NaN,Navy Fence (A),Coronado north beach,32.685586,-117.194319,150.0,Feet Downcoast,150.0,Feet Upcoast,...,NaN,0,0,0,NaN,2025-03-22,2025-03-23 00:00:00.000000,POINT (-117.19432 32.68559),place,2025-03-17
1,65567,NaN,Vista De La Playa,Marine Street Beach,32.834100,-117.281500,150.0,Feet Downcoast,150.0,Feet Upcoast,...,NaN,0,0,0,NaN,2025-03-15,2025-03-17 00:00:00.000000,POINT (-117.2815 32.8341),place,2025-03-10
1,65567,NaN,Vista De La Playa,Marine Street Beach,32.834100,-117.281500,150.0,Feet Downcoast,150.0,Feet Upcoast,...,NaN,0,0,0,NaN,2025-03-15,2025-03-17 00:00:00.000000,POINT (-117.2815 32.8341),place,2025-03-17
2,65565,NaN,Avenida Lunar,Coronado City beaches,32.674066,-117.172300,150.0,Feet Downcoast,150.0,Feet Upcoast,...,NaN,0,0,0,NaN,2025-03-14,2025-03-23 00:00:00.000000,POINT (-117.1723 32.67407),place,2025-03-10
2,65565,NaN,Avenida Lunar,Coronado City beaches,32.674066,-117.172300,150.0,Feet Downcoast,150.0,Feet Upcoast,...,NaN,0,0,0,NaN,2025-03-14,2025-03-23 00:00:00.000000,POINT (-117.1723 32.67407),place,2025-03-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,65219,NaN,Border Fence N side,Border Field State Park,32.535950,-117.124300,1000.0,Feet Upcoast,1000.0,Feet Downcoast,...,NaN,1,0,0,Tijuana Slough Shoreline. This Closure is a co...,2025-01-01,2025-04-10 09:43:03.438182,POINT (-117.1243 32.53595),place,2025-03-10
26,65219,NaN,Border Fence N side,Border Field State Park,32.535950,-117.124300,1000.0,Feet Upcoast,1000.0,Feet Downcoast,...,NaN,1,0,0,Tijuana Slough Shoreline. This Closure is a co...,2025-01-01,2025-04-10 09:43:03.438182,POINT (-117.1243 32.53595),place,2025-03-17
26,65219,NaN,Border Fence N side,Border Field State Park,32.535950,-117.124300,1000.0,Feet Upcoast,1000.0,Feet Downcoast,...,NaN,1,0,0,Tijuana Slough Shoreline. This Closure is a co...,2025-01-01,2025-04-10 09:43:03.438182,POINT (-117.1243 32.53595),place,2025-03-24
26,65219,NaN,Border Fence N side,Border Field State Park,32.535950,-117.124300,1000.0,Feet Upcoast,1000.0,Feet Downcoast,...,NaN,1,0,0,Tijuana Slough Shoreline. This Closure is a co...,2025-01-01,2025-04-10 09:43:03.438182,POINT (-117.1243 32.53595),place,2025-03-31


In [40]:

# Now, group by the 'week' column and count the number of events overlapping that week
weekly_counts = df_exploded.groupby('week').size()

print("Weekly counts (exploded method):")
print(weekly_counts)

Weekly counts (exploded method):
week
2024-12-30     7
2025-01-06     5
2025-01-13    11
2025-01-20     5
2025-01-27     9
2025-02-03    10
2025-02-10    10
2025-02-17     9
2025-02-24     6
2025-03-03     6
2025-03-10    10
2025-03-17    11
2025-03-24     6
2025-03-31     6
2025-04-07     5
dtype: int64
